In [1]:
!pip install pyspark

In [1]:
!rm -rf output_report/

In [3]:
from pyspark.sql import SparkSession

import os

from pyspark.sql import functions as f
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import collect_set, to_json, struct
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType

trips_schema = StructType([
    StructField('vendor_id', StringType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('ratecode_id', IntegerType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('pulocation_id', IntegerType(), True),
    StructField('dolocation_id', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType()),
])


def foreach_batch_function(df: DataFrame, epoch_id): # функция для записи батчей
    df.write.mode("append").json("output_report") # датафрейм будет писаться в папку output_report методом append


def main(spark: SparkSession):
    jsonOptions = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"} # указываем опции json-сериализации

    fields = list(map(lambda x: f"json_message.{x.name}", trips_schema.fields)) 

    df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "taxi") \
        .option("startingOffsets", "earliest") \
        .option("maxOffsetsPerTrigger", 1000) \
        .load() \
        .select(f.from_json(f.col("value").cast("string"), trips_schema, jsonOptions).alias("json_message")) \
        .select(fields)

    # пишем на диск
    writer = df \
        .writeStream \
        .foreachBatch(foreach_batch_function) \
        .trigger(processingTime='10 seconds') \
        .option("path", "output_report") \
        .outputMode("append") \
        .start()

    writer.awaitTermination()

In [5]:
main(
    SparkSession \
    .builder \
    .config("spark.driver.host", "localhost")\
    .appName("streaming_job") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
)

:: loading settings :: url = jar:file:/Users/popov-ilya/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/popov-ilya/.ivy2/cache
The jars for the packages stored in: /Users/popov-ilya/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e7d8396f-0e7c-4995-9ef3-7739a2858509;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 

CodeCache: size=131072Kb used=32666Kb max_used=32680Kb free=98405Kb
 bounds [0x0000000106d6c000, 0x0000000108d8c000, 0x000000010ed6c000]
 total_blobs=11851 nmethods=10851 adapters=913
 compilation: disabled (not enough contiguous free space left)


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/popov-ilya/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=80>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/popov-ilya/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/popov-ilya/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/popov-ilya/anaconda3/

Py4JError: An error occurred while calling o81.awaitTermination

In [7]:
from pyspark.sql import SparkSession

import os

from pyspark.sql import functions as f
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import collect_set, to_json, struct
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType

trips_schema = StructType([
    StructField('vendor_id', StringType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('ratecode_id', IntegerType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('pulocation_id', IntegerType(), True),
    StructField('dolocation_id', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType()),
])


def foreach_batch_function(df: DataFrame, epoch_id): # функция для записи батчей
    df.write.mode("append").json("output_report") # датафрейм будет писаться в папку output_report методом append


def main(spark: SparkSession):
    jsonOptions = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"} # указываем опции json-сериализации

    fields = list(map(lambda x: f"json_message.{x.name}", trips_schema.fields)) 

    df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "taxi") \
        .option("startingOffsets", "latest") \
        .option("maxOffsetsPerTrigger", 1000) \
        .load() \
        .select(f.from_json(f.col("value").cast("string"), trips_schema, jsonOptions).alias("json_message")) \
        .select(fields)
    
    mart = df \
        .groupBy('payment_type', f.to_date('tpep_pickup_datetime').alias('dt')) \
        .agg(f.count(f.col('*')).alias('cnt')) \
        .select(f.col('payment_type'), f.col('cnt'), f.col('dt')) \
        .orderBy(f.col('payment_type'), f.col('dt')) \
        .select(to_json(struct("payment_type", "cnt", "dt")).alias('value'))

    writer = mart \
        .writeStream \
        .trigger(processingTime='10 seconds') \
        .format("console") \
        .outputMode("complete") \
        .option("truncate", "false") \
        .start()

    writer.awaitTermination()

In [9]:
main(
    SparkSession \
    .builder \
    .config("spark.driver.host", "localhost")\
    .appName("streaming_job") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
)

25/02/22 10:43:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/l5/w530bl7d16l85pysj7rjxpk4cqhmc1/T/temporary-f8392c72-9d35-49d0-9b8c-80d487a41bd3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/02/22 10:43:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/02/22 10:43:28 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/22 10:43:28 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/22 10:43:28 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/22 10:43:28 WARN AdminClientConfig: The configuration '

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------------------------------+
|value                                       |
+--------------------------------------------+
|{"payment_type":1,"cnt":1,"dt":"2020-04-01"}|
+--------------------------------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------------------------------------------+
|value                                        |
+---------------------------------------------+
|{"payment_type":1,"cnt":58,"dt":"2020-04-01"}|
|{"payment_type":2,"cnt":33,"dt":"2020-04-01"}|
|{"payment_type":3,"cnt":1,"dt":"2020-04-01"} |
+---------------------------------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----------------------------------------------+
|value                                         |
+----------------------------------------------+
|{"payment_type":1,"cnt":1,"dt":"2020-03-08"}  |
|{"payment_type":1,"cnt":116,"dt":"2020-04-01"}|
|{"payment_type":2,"cnt":88,"dt":"2020-04-01"} |
|{"payment_type":3,"cnt":1,"dt":"2020-04-01"}  |
+----------------------------------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----------------------------------------------+
|value                                         |
+----------------------------------------------+
|{"payment_type":1,"cnt":1,"dt":"2020-03-08"}  |
|{"payment_type":1,"cnt":228,"dt":"2020-04-01"}|
|{"payment_type":2,"cnt":138,"dt":"2020-04-01"}|
|{"payment_type":3,"cnt":3,"dt":"2020-04-01"}  |
|{"payment_type":4,"cnt":1,"dt":"2020-04-01"}  |
+----------------------------------------------+



...

-------------------------------------------
Batch: 6
-------------------------------------------
+----------------------------------------------+
|value                                         |
+----------------------------------------------+
|{"payment_type":1,"cnt":4,"dt":"2020-02-03"}  |
|{"payment_type":1,"cnt":4,"dt":"2020-03-08"}  |
|{"payment_type":1,"cnt":501,"dt":"2020-04-01"}|
|{"payment_type":2,"cnt":2,"dt":"2020-02-03"}  |
|{"payment_type":2,"cnt":2,"dt":"2020-03-08"}  |
|{"payment_type":2,"cnt":371,"dt":"2020-04-01"}|
|{"payment_type":3,"cnt":9,"dt":"2020-04-01"}  |
|{"payment_type":4,"cnt":2,"dt":"2020-04-01"}  |
+----------------------------------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+----------------------------------------------+
|value                                         |
+----------------------------------------------+
|{"payment_type":1,"cnt":5,"dt":"2020-02-03"}  |
|{"payment_type":1,"cnt":4,"dt":"2020-03-08"}  |
|{"payment_type":1,"cnt":566,"dt":"2020-04-01"}|
|{"payment_type":1,"cnt":21,"dt":"2020-04-02"} |
|{"payment_type":2,"cnt":3,"dt":"2020-02-03"}  |
|{"payment_type":2,"cnt":4,"dt":"2020-03-08"}  |
|{"payment_type":2,"cnt":430,"dt":"2020-04-01"}|
|{"payment_type":2,"cnt":20,"dt":"2020-04-02"} |
|{"payment_type":3,"cnt":11,"dt":"2020-04-01"} |
|{"payment_type":4,"cnt":4,"dt":"2020-04-01"}  |
+----------------------------------------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+----------------------------------------------+
|value                                         |
+----------------------------------------------+
|{"payment_type":1,"cnt":9,"dt":"2020-02-03"}  |
|{"payment_type":1,"cnt":4,"dt":"2020-03-08"}  |
|{"payment_type":1,"cnt":566,"dt":"2020-04-01"}|
|{"payment_type":1,"cnt":90,"dt":"2020-04-02"} |
|{"payment_type":2,"cnt":3,"dt":"2020-02-03"}  |
|{"payment_type":2,"cnt":4,"dt":"2020-03-08"}  |
|{"payment_type":2,"cnt":431,"dt":"2020-04-01"}|
|{"payment_type":2,"cnt":112,"dt":"2020-04-02"}|
|{"payment_type":3,"cnt":11,"dt":"2020-04-01"} |
|{"payment_type":3,"cnt":3,"dt":"2020-04-02"}  |
|{"payment_type":4,"cnt":4,"dt":"2020-04-01"}  |
|{"payment_type":4,"cnt":1,"dt":"2020-04-02"}  |
+----------------------------------------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/popov-ilya/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/popov-ilya/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/popov-ilya/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [11]:
from pyspark.sql import SparkSession

import os

from pyspark.sql import functions as f
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import collect_set, to_json, struct
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType

trips_schema = StructType([
    StructField('vendor_id', StringType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('ratecode_id', IntegerType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('pulocation_id', IntegerType(), True),
    StructField('dolocation_id', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType()),
])


def foreach_batch_function(df: DataFrame, epoch_id): # функция для записи батчей
    df.write.mode("append").json("output_report") # датафрейм будет писаться в папку output_report методом append


def main(spark: SparkSession):
    jsonOptions = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"} # указываем опции json-сериализации

    fields = list(map(lambda x: f"json_message.{x.name}", trips_schema.fields)) 

    df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "taxi") \
        .option("startingOffsets", "latest") \
        .option("maxOffsetsPerTrigger", 1000) \
        .load() \
        .select(f.from_json(f.col("value").cast("string"), trips_schema, jsonOptions).alias("json_message")) \
        .select(fields)
    
    mart = df \
        .groupBy('payment_type', f.to_date('tpep_pickup_datetime').alias('dt')) \
        .agg(f.count(f.col('*')).alias('cnt')) \
        .select(f.col('payment_type'), f.col('cnt'), f.col('dt')) \
        .orderBy(f.col('payment_type'), f.col('dt')) \
        .select(to_json(struct("payment_type", "cnt", "dt")).alias('value'))

    writer = mart \
        .writeStream \
        .trigger(processingTime='10 seconds') \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:29092") \
        .option("topic", "report") \
        .option("checkpointLocation", "streaming-job-checkpoint") \
        .outputMode("complete") \
        .start()

    writer.awaitTermination()

In [13]:
main(
    SparkSession \
    .builder \
    .config("spark.driver.host", "localhost")\
    .appName("streaming_job") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
)

25/02/22 10:46:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/02/22 10:46:03 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/22 10:46:03 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/22 10:46:03 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/22 10:46:03 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/22 10:46:03 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
25/02/22 10:46:03 ERROR MicroBatchExecution: Query [id = 98a332bb-a401-47d2-b039-a248587cea37, runId = 794f39e5-636d-46d5-8dce-055ff52df376] terminated with error
java.lang.NoSuchMethodError: org.apache.spark.sql.types.StructType.toAttributes()Lscala/collection

StreamingQueryException: [STREAM_FAILED] Query [id = 98a332bb-a401-47d2-b039-a248587cea37, runId = 794f39e5-636d-46d5-8dce-055ff52df376] terminated with exception: org.apache.spark.sql.types.StructType.toAttributes()Lscala/collection/Seq;

In [17]:
spark.stop()

-------------------------------------------
Batch: 19
-------------------------------------------
+-----------------------------------------------+
|value                                          |
+-----------------------------------------------+
|{"payment_type":1,"cnt":9,"dt":"2020-02-03"}   |
|{"payment_type":1,"cnt":5,"dt":"2020-03-08"}   |
|{"payment_type":1,"cnt":823,"dt":"2020-04-01"} |
|{"payment_type":1,"cnt":1024,"dt":"2020-04-02"}|
|{"payment_type":2,"cnt":5,"dt":"2020-02-03"}   |
|{"payment_type":2,"cnt":5,"dt":"2020-03-08"}   |
|{"payment_type":2,"cnt":554,"dt":"2020-04-01"} |
|{"payment_type":2,"cnt":649,"dt":"2020-04-02"} |
|{"payment_type":3,"cnt":14,"dt":"2020-04-01"}  |
|{"payment_type":3,"cnt":16,"dt":"2020-04-02"}  |
|{"payment_type":4,"cnt":4,"dt":"2020-04-01"}   |
|{"payment_type":4,"cnt":12,"dt":"2020-04-02"}  |
+-----------------------------------------------+



NameError: name 'spark' is not defined